## 套件匯入

In [ ]:
import requests # HTTP 請求工具
from lxml import etree # 資料解析工具
from time import sleep # 休息函式

## HTTP 請求參數與鏈接

In [ ]:
KEYWORD = '誹謗'

link = 'https://law.moj.gov.tw/Law/LawSearchResult.aspx'

params = {
    'ty' : 'ONEBAR',
    'kw' : f'{KEYWORD}'
}

headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
}

## 發送 HTTP 請求

In [ ]:
res = requests.get(link,headers=headers,params=params) # 開始請求
print(res.status_code)

## 資料解析

In [ ]:
raw_data = etree.HTML(res.content.decode()) # 初始化套件並套用函式

In [ ]:
# 開始截取所需資料
data = [] # 用於儲存解析後資料
LINK_FORMAT = link[:link.index('tw')+2] # 截取主鏈接

for each in raw_data.xpath('//tbody//tr'): # 定位爬取項目中的大框架
    data.append({
        'link' : LINK_FORMAT+''.join(each.xpath('td[2]/a/@href'))[2:],
        'date' : ''.join(each.xpath('td[2]/text()')).strip().replace('(','').replace(')',''),
        'particular' : ''.join(each.xpath('td[2]/a/text()'))
    }) # 將需要的資料進行資料結構設計

## 內文爬蟲

In [ ]:
for each_p in data: # 針對所爬下來的項目進行內文爬蟲，因此需要 data 中的 link 資料
    res_each = requests.get(each_p['link'],headers=headers) # 針對每個 link 發出請求
    if res_each.status_code == 200: # 若請求成功則進行下一環節
        raw_each_data = etree.HTML(res_each.content.decode()) # 初始化套件並套用函式
        rule = [] # 用於儲存內文中所需的內容
        for each in raw_each_data.xpath("//div[@class='law-reg-content']/div"): # 定位爬取項目中的大框架
            rule.append({
                'rule_no' : ''.join(each.xpath('div/a/text()')),
                'rule_content' : ''.join(each.xpath('div/div/text()'))
            }) # 將需要的資料進行資料結構設計
        each_p['rule'] = rule # 將解析後的內文用 dict 的形式新增到 data 變數裡
    sleep(2) # 爬蟲的禮貌，好好休息

## 資料匯出 / 顯示爬取資料

In [ ]:
data